## Monitor transactions using anomaly detection model.     
---
**NOTE**: 

In real life scenarios financial transaction are dynamically evolving graphs. Performing anomaly detection inference on graph embeddings in live Transaction Monitoring Systems will require to update the graph and node representations after new transactions arrive. Recomputing entire graph for every newly arrived transaction will lead to unaxeptable delayes and even monitoring system failure. This problem  will be more sever if large amount of updates happen in a short time window.

Contact us at Logical Clocks and we will help you to setup end to end graph based deep anomaly detection for live Transaction Monitoring Systems. 

---

In [1]:
spark

Starting Spark application


ID,Application ID,Kind,State,Spark UI,Driver log
9,application_1651410992845_0010,pyspark,idle,Link,Link


SparkSession available as 'spark'.

In [2]:
import json
import numpy as np

![Image7-Monitor.png](./images/model_registry.gif)

# Query Model Repository for best anomaly detection model

In [3]:
import hsml

conn = hsml.connection()
mr = conn.get_model_registry()

Connected. Call `.close()` to terminate connection gracefully.

In [4]:
MODEL_NAME="ganAml"
EVALUATION_METRIC="loss"

In [5]:
best_model = mr.get_best_model(MODEL_NAME, EVALUATION_METRIC, "min")

In [6]:
print('Model name: ' + best_model.name)
print('Model version: ' + str(best_model.version))
print(best_model.training_metrics)

Model name: ganAml
Model version: 1
{'loss': '-0.2515300512313843'}

### Create the deployment
Here, we fetch the model we want from the model registry and define a configuration for the deployment. For the configuration, we need to specify the serving type (default or KFserving) and in this case, since we use default serving and an sklearn model, we need to give the location of the prediction script.

In [1]:
import hsml

conn = hsml.connection()
mr = conn.get_model_registry()

Starting Spark application


ID,Application ID,Kind,State,Spark UI,Driver log
10,application_1651410992845_0011,pyspark,idle,Link,Link


SparkSession available as 'spark'.
Connected. Call `.close()` to terminate connection gracefully.

In [10]:
# Use the model name from notebook 4, where we registered the model
model = mr.get_model(best_model.name, version=best_model.version)


In [19]:
# Create serving instance
SERVING_NAME = "ganTfServing"

# Give it any name you want
model.deploy(name=SERVING_NAME, model_server="TENSORFLOW_SERVING", serving_tool="DEFAULT")

## Fetch model server object 

In [2]:
ms = conn.get_model_serving()
deployment = ms.get_deployment('ganTfServing')

### Check Model Serving for active deployments
![Image7-Monitor.png](./images/deployment.gif)

In [ ]:
# Get serving status
#deployment.get_state()

In [ ]:
# Get serving status
#deployment.start()

# Retrieve serving vectors and send prediction requests to the served model using Hopsworks REST API

In [4]:
import hsfs
# Create a connection
connection = hsfs.connection()
# Get the feature store handle for the project's feature store
fs = connection.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

In [5]:
td_meta = fs.get_training_dataset("gan_non_sar_training_df", 1)
#`init_prepared_statement` method is needed to get serving_keys in case `get_serving_vector` has not beed called yet. This is not necessary for `get_serving_vector` method itself
td_meta.init_prepared_statement() 
td_meta.serving_keys

{'id'}

In [6]:
ids_to_score = ["0016359b", 
                "001dcc27", 
                "0054a022", 
                "00d6b609", 
                "00e14860", 
                "00e39a1b", 
                "014ed5cb", 
                "01ce3306", 
                "01fa19ae", 
                "01fa1d01", 
                "036dce03", 
                "03e09be4", 
                "04b23f4b"]

In [7]:
def model_server(input):
    data = {"inputs": [input]}
    return deployment.predict(data)

In [ ]:
for node_id in ids_to_score:
    serving_vector = td_meta.get_serving_vector({'id': node_id})
    print(serving_vector)
    model_server(serving_vector[0])

---